# 1. Allocation

In [ ]:
print('Running 1 key protocol')


DATA_LENGTH = int(KEY_LENGTH + np.ceil(np.log2(KEY_LENGTH)).astype(int) + 1)
Unprocessed_key_len = 3*DATA_LENGTH

# Preparation for encoding : Seed the random number generator. This will be used as our "coin flipper" 
random.seed(0)    

# order = np.ceil(np.log2(Unprocessed_key_len)).astype(int)
order = Order(Unprocessed_key_len)
dim = int(2**(order/2))

print(f"{Unprocessed_key_len = }, {KEY_LENGTH = }, {DATA_LENGTH = }, {order = }")

# 2. Alice

In [ ]:
# Generating a random string of bits
# If KEY_RESERVOIR(ALICE, BOB) exists, then alice_bits = KEY_RESERVOIR[:KEY_LENGTH]. Break the iteration here.

alice_bits = generate_random_bits(Unprocessed_key_len)
alice_bases = generate_random_bases(Unprocessed_key_len) # Alice randomly chooses a basis for each bit.

print(f" Alice uncorrected {block(alice_bits, order)}")

## 2.1 Quantum Encoding : 
**Encode the states into quantum circuits**

In [ ]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# 3. Quantum Signal Channel
This part can also be simulated by brute forcing 1-2 bit error at random.

## 3.1 Eve
0 : Not present,    1 : present

In [ ]:
if eve_presence == 'Random': eve = random.randint(0, 1)
else: eve = int(eve_presence)
    
label = 'Eve' if eve else 'Alice'
print(label)

In [ ]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]    # Initializing the circuit
errors_recorded = np.array([0, 0])    # Will keep track of the errors INJECTED deliberately by the algorithm

if eve : 
    #print("Eve Present!")
    qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
    
    errors_recorded = NoisyChannel(encoded_qubits, qubits_intercepted, 'Alice', errors_recorded, noise = ch_noise) ##Eve intercepts noisy states     

    eve_bases = generate_random_bases(Unprocessed_key_len) # Generate a random set of bases
    eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
    
    # Eve encodes her decoy qubits and sends them along the quantum channel    
    encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
    errors_recorded = NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', errors_recorded, noise = ch_noise) ## Eve sends noisy states to Bob

else : 
    errors_recorded = NoisyChannel(encoded_qubits, qubits_received, 'Alice', errors_recorded, noise = ch_noise) ## Alice sends noisy states to Bob


## 3.2 Bob

In [ ]:
bob_bases = generate_random_bases(Unprocessed_key_len) # Bob randomly chooses a basis for each bit.

# Measurement
bob_bits = measure(qubits_received, bob_bases)

print(f" {type(bob_bits[0]) = },  {type(alice_bits[0]) = }")

# 4. Public Interaction Channel

* On completion of this step, the length of bits will cut down to half of the original size.
* Alice can share a string suggesting in which order to use the received bits through QSC. 
* Alice can announce the PARITY_DICT (after sifting)

## 4.1 Sifting

In [ ]:
BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel

# Store the indices of the bases they share in common
common_bases = [i for i in range(Unprocessed_key_len) if bob_bases[i] == BROADCAST[i]]
bob_bits = [bob_bits[index] for index in common_bases]
bob_bits = ''.join(bob_bits)

BROADCAST = common_bases    # Bob tells Alice which bases they shared in common

alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common
alice_bits = ''.join(alice_bits)

print(f"\nAlice sent (& sifted) {block(alice_bits, order)} \n\nBob measured (& sifted){block(bob_bits, order)}")

## 4.2 Reconciliation 
(Comparision -- Spotting)
**Key size now reduced to half the original size**

In [1]:
# # What if Eve had listened to the previous round of key share and not this one? The remaining block would then be of mixed errors.
# # Therefore, if Eve is suspected, delete the entire lot of keys

# KEY_RESERVOIR_ALICE = np.concatenate(alice_bits[KEY_LENGTH])
# KEY_RESERVOIR_BOB = np.concatenate(bob_bits[KEY_LENGTH])

# alice_bits = KEY_RESERVOIR_ALICE[:KEY_LENGTH]
# bob_bits = KEY_RESERVOIR_BOB[:KEY_LENGTH]

# KEY_RESERVOIR_ALICE = KEY_RESERVOIR_ALICE[KEY_LENGTH:]
# KEY_RESERVOIR_BOB = KEY_RESERVOIR_BOB[KEY_LENGTH:]

In [ ]:
sample = len(alice_bits)//3    # len(alice_bits) >= 3
errors_detected = 0

for _ in range(sample):
    bit_index = random.randrange(len(alice_bits)) 
    
    if alice_bits[bit_index] != bob_bits[bit_index]:  errors_detected += 1    #calculating errors

    #removing tested bits from key strings
    alice_bits = alice_bits[:bit_index] + alice_bits[bit_index+1 :] 
    bob_bits = bob_bits[:bit_index] + bob_bits[bit_index+1 :]

order = Order(alice_bits)

## 4.3 QBER

In [ ]:
print(f' Errors inflicted[bit, phase] : {errors_recorded}, Errors detected(total) : {errors_detected}, {sample = }')

In [ ]:
### QBER should be ~ 0.5 (instead of ~0.25) in presence of Eve, because the sample size is 1/3 of the bits AFTER sifting.

QBER = round(errors_detected/sample, 5) # calculating QBER and saving the answer to two decimal places
print(f"{QBER = }")
print(f"\n Error Threshold : {error_threshold}")

key = alice_bits
if QBER > error_threshold:
    # num_keys += 1
    raise RuntimeError('\n Eve{} detected'.format('' if eve else ' Falsely'))
    print(f" \n ABORTING and Restarting... \n\n\n\n")
    I = I-1
    flag = 0
    
else :
    print(" Key is secure: ", end = " ")
    flag = 1
    if eve : print(' Eve went unnoticed : ')
    else : print(' Eve not present : ')

    print(f" \n Proceeding towards Error-Correction... \n\n\n\n")
    # KEY_RESERVOIR = np.concatenate(KEY_RESERVOIR, alice_bits)

# 5. Updating the variables

In [ ]:
QBERs[I] = QBER
KEY_RESERVOIR.append(key)
KEY_RESERVOIR_len[I] = len(key)

Eve_detected[I] = ((QBER >= error_threshold) and eve) + ((QBER < error_threshold) and not eve)    # Whether or not the DETECTION of Eve is CORRECT